## Create Retrieval Dataset

In this notebook, we'll create a dataset for our retrieval model.

In [ ]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

### Feature Selection

First, we'll load the feature groups we created in the previous tutorial.

In [ ]:
trans_fg = fs.get_feature_group("transactions")
customers_fg = fs.get_feature_group("customers")
articles_fg = fs.get_feature_group("articles")

We'll need to join these three data sources to make the data compatible with out retrieval model. Recall that each row in the `transactions` feature group relates information about which customer bought which item. We'll join this feature group with the `customers` and `articles` feature groups to inject customer and item features into each row.

In [ ]:
query = trans_fg.select(["customer_id", "article_id", "month_sin", "month_cos"])\
    .join(customers_fg.select(["age"]), on="customer_id")\
    .join(articles_fg.select(["garment_group_name", "index_group_name"]), on="article_id")

### Transformation Functions
Transformation functions are a mathematical mapping of input data that may be stateful - requiring statistics from the parent feature view (such as number of instances of a category, or mean value of a numerical feature)

We will preprocess our data using min-max scaling on numerical features and label encoding on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as min-max scaling are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [ ]:
# Load transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformations.
transformation_functions = {
    "age" : min_max_scaler,
    # "garment_group_name" : label_encoder,
    # "index_group_name" : label_encoder
}

# TODO for some reason label_encoder seems to cause problems when doing td.read()?

### Feature View Creation
In Hopsworks, you write features to feature groups (where the features are stored) and you read features from feature views. A feature view is a logical view over features, stored in feature groups, and a feature view typically contains the features used by a specific model. This way, feature views enable features, stored in different feature groups, to be reused across many different models.

In [ ]:
feature_view = fs.create_feature_view(
    name='retrieval_fv',
    query=query,
    transformation_functions=transformation_functions
)

To view and explore data in the feature view we can retrieve batch data using the `get_batch_data()` method.

In [ ]:
feature_view.get_batch_data().head(5)

### Training Dataset Creation

Finally, we can create our dataset.

In [ ]:
# TODO we will use a chronological split instead.

td = feature_view.create_training_dataset(
    description = 'retrieval_dataset_splitted',
    data_format = 'csv',
    splits = {'train': 80, 'validation': 20},
    train_split = "train",
    write_options = {'wait_for_job': False}
)

### Next Steps

In the next notebook, we'll train a model on the dataset we created in this notebook.